In [1]:
import pandas as pd
import numpy  as np

In [53]:
# load in data, specify dtype of ambiguous data
df1 = pd.read_csv("data/JobOper.csv", encoding="latin1", dtype={'JobNum': str, 'PurPoint': str, 'WIName': str, 'AssemblySeq': str, 'OprSeq': str})
df2 = pd.read_csv("data/jobHead.csv", encoding="latin1", dtype={'JobNum': str, 'EngineerCode_c': str})
df3 = pd.read_csv("data/LaborDtl.csv", encoding="latin1", dtype={'JobNum': str, 'AssemblySeq': str, 'OprSeq': str})

In [8]:
# keep only completed Jobs
df1 = df1[df1.JobComplete == True]
df2 = df2[df2.JobComplete == True]

## Review Available fields 
Looking at three data sources, JobOper, JobHead and LaborDtl, we're going to take a look at all the available fields across these tables. Some names might stand out as being useful for building a feature set that can predict lead times.

In [9]:
cols = sorted(df1.columns)

print('JobOper Table\n------------')
for i in range(0, len(cols) - 5, 5):
    print('{:20}{:20}{:20}{:20}{:20}'.format(*cols[i:i+5]))

JobOper Table
------------
ActBurCost          ActLabCost          ActProdHours        AddedOper           AssemblySeq         
CommentText         Company             DaysOut             Description         DocUnitPrice        
DueDate             DueHour             EstProdHours        EstScrap            EstScrapType        
EstSetHours         EstSetHoursPerMch   EstUnitCost         HoursPerMachine     IUM                 
JobComplete         JobEngineered       JobNum              LaborEntryMethod    LastLaborDate       
LoadDate            LoadHour            Machines            MoveDueDate         MoveDueHour         
OpCode              OpComplete          OpDesc              OprSeq              OpsPerPart          
PartNum             PrimaryProdOpDtl    PrimarySetupOpDtl   ProdBurRate         ProdComplete        
ProdCrewSize        ProdLabRate         ProdStandard        PurPoint            QtyCompleted        
QtyPer              QueStartDate        QueStartHour        Revi

In [87]:
cols = sorted(df3.columns)

print('LaborDtl Table\n------------')
for i in range(0, len(cols) - 5, 5):
    print('{:20}{:20}{:20}{:20}{:20}'.format(*cols[i:i+5]))

LaborDtl Table
------------
ActiveTrans         ClockInDate         ClockInMInute       ClockOutMinute      ClockOutTime        
ClockinTime         Company             Complete            CreateDate          Downtime            
EarnedHrs           JobNum              LaborDtlSeq         LaborEntryMethod    LaborHedSeq         
LaborHrs            LaborQty            LaborRate           LaborType           OpCode              
OpComplete          OprSeq              ResourceGrpID       ResourceID          SetupPctComplete    


In [88]:
cols = sorted(df2.columns)

print('JobHead Table\n------------')
for i in range(0, len(cols) - 5, 5):
    print('{:20}{:20}{:20}{:20}{:20}'.format(*cols[i:i+5]))

JobHead Table
------------
AnalysisCode        BasePartNum         BaseRevisionNum     CallLine            CallNum             
Candidate           CheckOff1           CheckOff2           CheckOff3           CheckOff4           
CheckOff5           CloseMeterReading   ClosedDate          CommentText         Company             
ContractID          CreateDate          CreatedBy           CustID              CustName            
Customer_c          DatePurged          DaysLate            DrawNum             DtlsWithinLeadTime  
DueDate             DueHour             EPMExportLevel      EngineerCode_c      EngineeringApproval_c
EngineeringReview_c EquipID             ExpenseCode         ExportRequested     ExternalMES         
ForeignSysRowID     Forward             GroupSeq            HDCaseNum           IUM                 
InCopyList          InitialReview_c     IsCSRSet            IssueTopicID1       IssueTopicID10      
IssueTopicID2       IssueTopicID3       IssueTopicID4       Iss

### Create a few features from JobOper, JobHead and LaborDtl tables
From the JobHead table we can get the scheduled start and due date, as well as the actual completion dates of the jobs. There is no "Actual Start Date" in the job tables, so we're going to get this from production data (found in the LaborDtl table). This gives us a feature (the schedule-suggested lead time) and the outcome (the actual days to complete). These lead times are based on number of business days. 

In addtion, we'll pull a couple features from the JobOper table that aggregates production hours from all the operations on each job, as well as costs and subcontract lead times. These features describe the operations that are assigned to a job. 

In [63]:
# ERP Scheduled Lead Times
valid_rows = df2.dropna(axis=0, subset=['StartDate', 'DueDate']).index  # Drop jobs with missing schedules
sched_leadTime = df2.loc[valid_rows, ['JobNum', 'StartDate', 'DueDate']]
sched_leadTime['SchedDays'] = sched_leadTime.apply(lambda x: len(pd.bdate_range(x.StartDate, x.DueDate)), axis=1)
sched_leadTime = sched_leadTime.drop(labels=['StartDate', 'DueDate'], axis=1)

# Actual (Ground Truth) Lead Times
# First get the first labor detail creation date (first activity) and combine with the actual job completion date
first_activity = df3.loc[:, ['JobNum', 'CreateDate']].groupby('JobNum', as_index=False).apply(min)

# Remove data where we don't have the ground truth information available
job_completion = df2[['JobNum', 'JobCompletionDate']].dropna(subset=['JobCompletionDate'])
first_activity = first_activity.merge(job_completion, how='inner', on='JobNum')
del job_completion

# Calculate the Actual Lead time, drop the columns used for the calculation afterwards
first_activity['ActDays'] = first_activity.apply(lambda x: len(pd.bdate_range(x.CreateDate, x.JobCompletionDate)), axis=1)
first_activity = first_activity.drop(labels=['JobCompletionDate', 'CreateDate'], axis=1)

# Number of operations on the job
num_operations = df1.groupby('JobNum', as_index=False).size()
num_operations = num_operations.to_frame(name='OpCount').reset_index()

# Number of Production hours to complete
tot_production = df1.loc[:, ['JobNum', 'EstProdHours']].groupby('JobNum', as_index=False).sum()
tot_production = tot_production.rename(columns={'EstProdHours':'TotalProdHours'})

# Number of setup hours
tot_setupHours = df1.loc[:, ['JobNum', 'EstSetHours']].groupby('JobNum', as_index=False).sum()
tot_setupHours = tot_setupHours.rename(columns={'EstSetHours':'TotalSetupHours'})

# total lead time for all subcontract operations
tot_subleadtime = df1.loc[:, ['JobNum', 'DaysOut']].groupby('JobNum', as_index=False).sum()
tot_subleadtime = tot_subleadtime.rename(columns={'DaysOut':'TotalSubLeadTime'})

# total costs of subcontract operations
tot_subconCosts = df1.loc[:, ['JobNum', 'EstUnitCost']].groupby('JobNum', as_index=False).sum()
tot_subconCosts = tot_subconCosts.rename(columns={'EstUnitCost':'TotalSubCost'})

# number of subcontract operations
num_subcontract = df1.loc[:, ['JobNum', 'SubContract']].groupby('JobNum', as_index=False).sum()
num_subcontract = num_subcontract.rename(columns={'SubContract':'TotalSubContractOps'})

# Job Creation Date
job_createdates = df3.loc[:, ['JobNum', 'CreateDate']]

### Combine all features

In [75]:
# drop jobs without any schedule
df = df2.copy().loc[:, ['JobNum', 'Company']]

df = df.merge(sched_leadTime, how='left', on='JobNum')
df = df.merge(num_operations, how='left', on='JobNum')
df = df.merge(tot_production, how='left', on='JobNum')
df = df.merge(tot_setupHours, how='left', on='JobNum')
df = df.merge(tot_subleadtime, how='left', on='JobNum')
df = df.merge(tot_subconCosts, how='left', on='JobNum')
df = df.merge(num_subcontract, how='left', on='JobNum')
# df = df.merge(df3.loc[:, ['JobNum', 'CreateDate']], how='left', on='JobNum')
df = df.merge(first_activity, how='inner', on='JobNum')

df.head(10)

,JobNum,Company,SchedDays,OpCount,TotalProdHours,TotalSetupHours,TotalSubLeadTime,TotalSubCost,TotalSubContractOps,ActDays
0,70398,HEM,26.0,14.0,21.04,2.55,11.0,0.49593,3.0,45
1,80586,HEM,44.0,12.0,9.25,3.50,29.0,7.99764,4.0,42
2,68912,HEM,33.0,13.0,36.47,10.25,3.0,0.00000,1.0,43
3,80917,HEM,23.0,7.0,16.41,2.84,8.0,12.74412,2.0,14
4,80639,HEM,16.0,4.0,3.50,0.00,10.0,0.35000,1.0,10
5,70116,HEM,40.0,4.0,13.50,3.00,0.0,0.00000,0.0,55
6,021088-1,SMC,8.0,7.0,88.02,0.50,2.0,0.00000,1.0,5
7,021365,SMC,10.0,NaN,NaN,NaN,NaN,NaN,NaN,22
8,020659,SMC,24.0,NaN,NaN,NaN,NaN,NaN,NaN,82
9,81725,HEM,5.0,4.0,6.89,2.00,0.0,0.00000,0.0,2


In [76]:
print('Total Rows:', df.shape[0])

Total Rows: 4451


In [77]:
df.describe()

,SchedDays,OpCount,TotalProdHours,TotalSetupHours,TotalSubLeadTime,TotalSubCost,TotalSubContractOps,ActDays
count,4446.000000,3062.000000,3062.000000,3062.000000,3062.000000,3062.000000,3062.000000,4451.000000
mean,28.780927,6.776617,28.906721,2.413703,10.367469,4.747488,1.258328,38.975960
std,143.575166,4.197400,109.909899,3.155040,22.680122,19.663387,1.319403,47.741825
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,3.000000,3.300000,0.312500,0.000000,0.000000,0.000000,6.000000
50%,16.000000,6.000000,8.680000,2.000000,5.000000,0.018295,1.000000,25.000000
75%,33.000000,9.000000,22.685000,3.190000,13.000000,1.980023,2.000000,54.000000
max,9133.000000,32.000000,3939.110000,102.000000,535.000000,594.676900,10.000000,553.000000


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4451 entries, 0 to 4450
Data columns (total 10 columns):
JobNum                 4451 non-null object
Company                4451 non-null object
SchedDays              4446 non-null float64
OpCount                3062 non-null float64
TotalProdHours         3062 non-null float64
TotalSetupHours        3062 non-null float64
TotalSubLeadTime       3062 non-null float64
TotalSubCost           3062 non-null float64
TotalSubContractOps    3062 non-null float64
ActDays                4451 non-null int64
dtypes: float64(7), int64(1), object(2)
memory usage: 382.5+ KB


In [1]:
df = df.dropna()

NameError: name 'df' is not defined

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3058 entries, 0 to 4450
Data columns (total 10 columns):
JobNum                 3058 non-null object
Company                3058 non-null object
SchedDays              3058 non-null float64
OpCount                3058 non-null float64
TotalProdHours         3058 non-null float64
TotalSetupHours        3058 non-null float64
TotalSubLeadTime       3058 non-null float64
TotalSubCost           3058 non-null float64
TotalSubContractOps    3058 non-null float64
ActDays                3058 non-null int64
dtypes: float64(7), int64(1), object(2)
memory usage: 262.8+ KB


In [83]:
df.to_csv('data/features.csv', sep=',', encoding='utf-8', index=False)

In [82]:
df.head()

,JobNum,Company,SchedDays,OpCount,TotalProdHours,TotalSetupHours,TotalSubLeadTime,TotalSubCost,TotalSubContractOps,ActDays
0,70398,HEM,26.0,14.0,21.04,2.55,11.0,0.49593,3.0,45
1,80586,HEM,44.0,12.0,9.25,3.50,29.0,7.99764,4.0,42
2,68912,HEM,33.0,13.0,36.47,10.25,3.0,0.00000,1.0,43
3,80917,HEM,23.0,7.0,16.41,2.84,8.0,12.74412,2.0,14
4,80639,HEM,16.0,4.0,3.50,0.00,10.0,0.35000,1.0,10
